Flavor physics analysis looking for decays of D or B mesons. In this example we run the reconstruction of Ds->phi(mumu)pi decays, looking for two collimated muons and a charged track originating from a common vertex. The analysis is performed on ntuples produced running on CMS data in the MINIAOD format.

# Basic imports

In [ ]:
import sys, os, time
start = time.time()
import json
import ROOT
from ROOT import * 

# Dask scheduler

In [ ]:
from dask.distributed import Client

Now start new Dask cluster, scale the number of workers

In [ ]:
#fill this cell with "<>" button of Dask labextension

# Declare custom C++ functions 

In [ ]:
text_file = open("Utilities.h", "r")
data = text_file.read()


def my_initialization_function():
    ROOT.gInterpreter.Declare('{}'.format(data))

ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)

# X509 proxy configuration
The `/tmp/x509up_u` file should be generated prior running the notebook using `voms-proxy-init `

In [ ]:
from distributed.diagnostics.plugin import UploadFile
Client.register_worker_plugin(UploadFile("/tmp/x509up_u0"))

In [ ]:
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'x509up_u0'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR") 

In [ ]:
client.run(set_proxy)

# Define chain of rootfiles to analyze
ntuples corresponding to different datasets and eras are defined in a configuration json file

In [ ]:
configjson = "/opt/workspace/persistent-storage/BPH_usecase/input.json"
dataset_to_process = "ParkingBPH1_D" #"DsToNMu_NToMuPi_mN1p5_ctau1000p0mm_incl"

with open(configjson, "r") as f:
    config = json.loads(f.read())
    
print(config)